# **Musculoskeletal Radiographs Abnormality Detection**

# **Exploratory Data Analysis**

### **Problem**

Determining whether a radiographic study is normal or abnormal is a critical radiological task: a study interpreted as normal rules out disease and can eliminate the need for patients to undergo further diagnostic procedures or interventions. The musculoskeletal abnormality detection task is particularly critical as more than 1.7 billion people are affected by musculoskeletal conditions worldwide (BMU, 2017). These conditions are the most common cause of severe, long-term pain and disability (Woolf & Pfleger, 2003), with 30 million emergency department visits annually and increasing. Our dataset, MURA, contains 9,045 normal and 5,818 abnormal musculoskeletal radiographic studies of the upper extremity including the shoulder, humerus, elbow, forearm, wrist, hand, and finger. MURA is one of the largest public radiographic image datasets. More information about the  the dataset can be found [here](http://stanfordmlgroup.github.io/competitions/mura/)  as well as [in this paper](http://https://arxiv.org/pdf/1712.06957.pdf)

# Libraries

In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from enum import Enum
import cv2
from tensorflow.keras import backend as K # Importing Keras backend (by default it is Tensorflow)
import gc
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

ModuleNotFoundError: ignored

# Load data

This data comes as split dataset (training & validation in separate directories). Each data categories such as training & validation has sub-directories (pls see below).


```

└─train {data category}
│   └───XR_ELBOW {study type}
│       │  └───patient00011 {patient}
│       │         └───study1_negative {study with label}
│       │               └───image1.png {radiographs}
│       │               └───image2.png
│       │               └───image3.png
                        └───...
   ...
   

└─valid {data category}
│   └───XR_HUMERUS {study type}
│       │  └───patient11216 {patient}
│       │         └───study1_negative {study with label}
│       │               └───image1.png {radiographs}
│       │               └───image2.png
                        └───...
```

## First we Download the dataset

In [ ]:
"""
%%bash
wget 'https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip'
unzip MURA-v1.1.zip
"""

#### Let us load the Dataset

In [ ]:
cd ../input/mura-v11

##### Helper Function for loading dataset

In [ ]:
def load_path(path):
    dataset = []
    for body in os.listdir(path):
        body_part = body

        path_p = path+'/'+str(body)

        for id_p in os.listdir(path_p):
            patient_id = id_p
            path_id = path_p+'/'+str(id_p)
            for lab in os.listdir(path_id):
                if lab.split('_')[-1]=='positive':
                    label = 1
                elif lab.split('_')[-1]=='negative':
                    label= 0

                path_l = path_id+'/'+str(lab)

                for img in os.listdir(path_l):
                    img_path = path_l + '/' + str(img)

                    dataset.append(
                        {
                            'body_part': body_part,
                            'patient_id': patient_id,
                            'label': label,
                            'img_path': img_path
                        }
                    )
    return dataset

In [ ]:
path = 'MURA-v1.1/train'
os.listdir(path)

In [ ]:
dataset = load_path(path)
df_train = pd.DataFrame(dataset)
df_train.head(2)

In [ ]:
dataset_test = load_path(path = 'MURA-v1.1/valid')
df_test = pd.DataFrame(dataset_test)

## Data Preprocessing

#### Creating Dataset for the Model traing and generating Labels for the same.

In [ ]:
df_train['label_index']= df_train.label
df_train.label_index.replace('positive', 1, inplace=True)
df_train.label_index.replace('negative', 0, inplace=True)
df_train.head(3)

In [ ]:
df_test['label_index']= df_test.label
df_test.label_index.replace('positive', 1, inplace=True)
df_test.label_index.replace('negative', 0, inplace=True)

## Exploratory Data Analysis

In [ ]:
print('We have '+str(len(df_train))+' train images from all body parts')

In [ ]:
print("Total Number of training images : ", len(df_train["img_path"]))

In [ ]:
print ("\n\nnumber of null values in:\n", df_train.isnull().sum())

In [ ]:
categories_counts = pd.DataFrame(df_train['body_part'].value_counts())
print ('\n\ncategories:\n',categories_counts )

In [ ]:
print('\n\nnumber of patients:',df_train['patient_id'].nunique())

In [ ]:
print('\n\nNumber of different labels:',df_train['label'].nunique())

#### Different Catagories of Body in which we are checking for abnormality

In [ ]:
print('We have Bone X-Ray from: ')
path = 'MURA-v1.1/train'
print(*os.listdir(path), sep = "\n")

##### Helper Function for Visualization of Dataset

In [ ]:
def count_distrib(df_body, body_name):
    grouped = df_body.groupby(df_body.label)
    pos = len(grouped.get_group(1))
    print('We have ' +str(pos)+ ' positive with abnormality '+ body_name + ' scans')
    neg = len(grouped.get_group(0))
    print( 'We have ' +str(neg)+ ' negative with abnormality '+ body_name + ' scans')
    total = len(df_body)
    return body_name, pos, neg, total

#### Exploring Different Catagories of Body in which we are checking for abnormality

### 1. Elbow Joint

In [ ]:
grouped = df_train.groupby(df_train.body_part)
df_XR_ELBOW = grouped.get_group("XR_ELBOW")
df_XR_ELBOW.head(4)

In [ ]:
print("Number of training Examples for ELBOW Joint : ",len(df_XR_ELBOW))

In [ ]:
training_data_distr = []

In [ ]:
body_name, pos, neg, total = count_distrib(df_XR_ELBOW, 'XR_ELBOW')

training_data_distr.append(
                    {
                    'body_part': body_name,
                    'positive': pos,
                    'negative': neg,
                    'total':total
                    })

### 2. Finger Joint

In [ ]:
df_XR_FINGER = grouped.get_group("XR_FINGER")


In [ ]:
print("Number of training XR_FINGER: ",len(df_XR_FINGER))

In [ ]:
body_name, pos, neg, total = count_distrib(df_XR_FINGER, 'XR_FINGER')

training_data_distr.append(
                    {
                    'body_part': body_name,
                    'positive': pos,
                    'negative': neg,
                    'total':total
                    })

### 3. Forearm Joint

In [ ]:
df_XR_FOREARM = grouped.get_group("XR_FOREARM")

In [ ]:
print("Number of training XR_FOREARM: ",len(df_XR_FOREARM))

In [ ]:
body_name, pos, neg, total = count_distrib(df_XR_FOREARM, 'XR_FOREARM')

training_data_distr.append(
                    {
                    'body_part': body_name,
                    'positive': pos,
                    'negative': neg,
                    'total':total
                    })

### 4. Hand Joint

In [ ]:
df_XR_HAND = grouped.get_group("XR_HAND")

In [ ]:
print("Number of training XR_HAND: ",len(df_XR_HAND))

In [ ]:
body_name, pos, neg, total = count_distrib(df_XR_HAND, 'XR_HAND')

training_data_distr.append(
                    {
                    'body_part': body_name,
                    'positive': pos,
                    'negative': neg,
                    'total':total
                    })

### 5. Humerus Joint

In [ ]:
df_XR_HUMERUS = grouped.get_group("XR_HUMERUS")

In [ ]:
print("Number of training XR_HUMERUS: ",len(df_XR_HUMERUS))

In [ ]:
body_name, pos, neg, total = count_distrib(df_XR_HUMERUS, 'XR_HUMERUS')

training_data_distr.append(
                    {
                    'body_part': body_name,
                    'positive': pos,
                    'negative': neg,
                    'total':total
                    })

### 6. Shoulder Joint

In [ ]:
df_XR_SHOULDER = grouped.get_group("XR_SHOULDER")

In [ ]:
print("Number of training XR_SHOULDER: ",len(df_XR_SHOULDER))

In [ ]:
body_name, pos, neg, total = count_distrib(df_XR_SHOULDER, 'XR_SHOULDER')

training_data_distr.append(
                    {
                    'body_part': body_name,
                    'positive': pos,
                    'negative': neg,
                    'total':total
                    })

### 7. Wrist Joint

In [ ]:
df_XR_WRIST = grouped.get_group("XR_WRIST")

In [ ]:
print("Number of training XR_WRIST: ",len(df_XR_WRIST))

In [ ]:
body_name, pos, neg, total = count_distrib(df_XR_WRIST, 'XR_WRIST')

training_data_distr.append(
                    {
                    'body_part': body_name,
                    'positive': pos,
                    'negative': neg,
                    'total':total
                    })

### Explore data distribution with plots

In [ ]:
training_data_distr = pd.DataFrame(training_data_distr)

In [ ]:
training_data_distr['percentage'] = round(100 * training_data_distr['total'] / len(df_train.body_part), 2)

#### The Overview of our Training Dataset

In [ ]:
print('Training Data Distribution')
training_data_distr

In [ ]:
labels = training_data_distr['body_part']
negative = training_data_distr['negative']
positive = training_data_distr['positive']

x = np.arange(len(labels))
width = 0.25

fig = plt.figure(figsize=(14, 10))

ax = fig.subplots()
rects1 = ax.bar(x - width/2, negative, width, label='Negative')
rects2 = ax.bar(x + width/2, positive, width, label='Positive')
ax.set_ylabel('Number of Body Parts')
ax.set_xlabel('Body Parts')
ax.set_title('Counts distribution of X-Ray fro each part of body')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.00*height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

plt.show()

### Observation from the graph is -
* We have the least data for Humerus Joint followed by Forearm.
* We have the largest data for Wrist and Shoulder.
* This variation in the data point will make it more harder for the model to learn about Abnormality in Humerus and Forearm joint.


### Let us analyze dataset more

##### 1. Training Set

In [ ]:
grouped = df_train.groupby(df_train.label)
pos = len(grouped.get_group(1))
neg = len(grouped.get_group(0))

In [ ]:
print("Number of positives:",pos)
print("Number of negatives:",neg)

In [ ]:
# the data you want to plot
categories = ["Negative"]
values     = [ neg]

categories2 = ["Positive"]
values2     = [  pos]

plt.bar(categories, values)
plt.bar(categories2, values2)
plt.title('Distribution of X-Ray Images')
plt.xlabel('Detection')
plt.ylabel('count')


plt.show()

#### We can conclude that the dataset of trainnig data is unbalance with more negative images.

#### 2. Test set

In [ ]:
df_test.label.head()

In [ ]:
grouped = df_train.groupby(df_train.label)
pos = len(grouped.get_group(1))
neg = len(grouped.get_group(0))

In [ ]:
print("Number of positives:",pos)
print("Number of negatives:",neg)

In [ ]:
categories = ["Negative"]
values     = [ neg]
categories2 = ["Positive"]
values2     = [  pos]
plt.bar(categories, values)
plt.bar(categories2, values2)
plt.title('Distribution of X-Ray')
plt.xlabel('labels')
plt.ylabel('count')
plt.show()

#### The dataset of test data is far more balance than train data.


## Now Preprocessing some Images .

In [ ]:
im_size = 256

#### Random Rotation flip

In [ ]:
def random_rotation_flip(image,size = 256):
    if random.randint(0,1):
        image = cv2.flip(image,1) # 1-->horizontal flip 0-->Vertical flip -1-->Horizontal and vertical

    if random.randint(0,1):
            angle = random.randint(-30,30)
            M = cv2.getRotationMatrix2D((size/2,size/2),angle,1)
            #The third parameter: the size of the transformed image
            image = cv2.warpAffine(image,M,(size,size))
    return image

In [ ]:
def image_loader(Path, size = 224):

    Images = []

    for path in tqdm(Path):
        try:
            image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            image = random_rotation_flip(image,size)
            Images.append(image)

        except Exception as e:
            print(str(e))

    Images = np.asarray(Images).astype('float32')

    #normalization
    mean = np.mean(Images)
    std = np.std(Images)
    Images = (Images - mean) / std

    return Images

In [ ]:
df_train.head()

In [ ]:
cd ../input/mura-v11

In [ ]:
X_train = image_loader(df_train['img_path'][:50,],im_size)

In [ ]:
y_train = df_train['label']
Y_train = y_train.replace("positive",1)
Y_train = Y_train.replace("negative",0)

In [ ]:
X_test = image_loader(df_test['img_path'][:50,],im_size)

In [ ]:
y_test = df_test['label']
Y_test = y_test.replace("positive",1)
Y_test = Y_test.replace("negative",0)

### Plots/ Visualization of Images.

In [ ]:
def print_img_with_colorbar(image):
    fig = plt.figure()
    plt.imshow(image, cmap = plt.cm.binary)
    plt.colorbar()

In [ ]:
print_img_with_colorbar(X_train[0])

In [ ]:
def print_range_images(images, images_label):
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel(images_label[i])

print_range_images(X_train, y_train)

In [ ]:
from skimage.io import imread
sub_df = df_train.groupby(['body_part', 'label']).apply(lambda x: x.sample(1)).reset_index(drop = True)
fig, (m_axs) = plt.subplots(4, sub_df.shape[0]//4, figsize = (12, 12))
for c_ax, (_, c_row) in zip(m_axs.flatten(), sub_df.iterrows()):
    c_ax.imshow(imread(c_row['img_path']), cmap = 'bone')
    c_ax.axis('off')
    c_ax.set_title('{body_part}:{label}'.format(**c_row))